In [109]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [110]:
all_property = []

for page in range(23,29):
    print('Page: ', page)
    link = 'https://www.apartments.com/houses/ca/' + str(page) + '/'
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0'
    headers = {'User-Agent': user_agent}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
#     print(soup)
#     page 1 - 3
#     results = soup.find_all('article', class_='placard placard-option has-header js-premiumplus')
#     page 3-23
#     results = soup.find_all('article', class_='placard placard-option no-header js-premium')
#     page
    results = soup.find_all('article', class_='placard placard-option no-header js-prosumer')

    # Loop through returned results and get the hyperlink of the properties of the page
#     print('results: ', results)
    for result in results:
#         print('result: ', result)
        find_tag = result.find('a', class_="property-link")['href']
        print(find_tag)
        new_response = requests.get(find_tag, headers=headers)
        new_soup = BeautifulSoup(new_response.text, 'html.parser')
        new_results = new_soup.find('div', class_="mainWrapper")
#     page 1 - 3
#         full_address = result.find('div', class_ = "property-information").text.strip().replace('\n', ' ')
#     page 3-28
        address = result.findAll('div', class_ = "js-url")
        full_address = ""
        for i in address:
            full_address = full_address + i.text + " "
        full_address = full_address.strip()
        print('full_address: ', full_address)
        city = new_results.find('div', class_ = "propertyAddressContainer").select('span')[1].text
#         print('city: ', city)
        state = new_results.find('div', class_ = "propertyAddressContainer").select('span')[3].text
#         print('state: ', state)
        zip_code = new_results.find('div', class_ = "propertyAddressContainer").select('span')[4].text
#         print('zip_code: ', zip_code)
        street = new_results.find('h1', class_ = "propertyName").text.replace('\r\n                    ', "").strip()
        bedroom_container = new_results.find('div', class_="rentRollupContainer")
        house_details = new_results.find('div', class_="priceBedRangeInfoContainer")
#         print('house_details: ', house_details)
        bedroom_container = new_results.find('section', class_="availabilitySection")
#         print('bedroom_container: ', bedroom_container)
        all_the_rents = bedroom_container.findAll('div', class_="priceBedRangeInfo")
#         print('all_the_rents: ', all_the_rents)
        
        rent_obj = {}
        for rent_block in all_the_rents:
#             print('test: ', rent_block)
            price = rent_block.find('span', class_ = "rentLabel").text.strip()
            if "–" in price:
                price = price.split()[0]
            if "Call for Rent" in price or "/" in price:
                continue
            float_price = float(price.replace("$", "").replace(",", ""))
#             print('rent_block: ', rent_block)
#             print('price: ', price)
            testing = rent_block.find('h4', class_ = "detailsLabel").find('span', class_ = "leaseDepositLabel").text
            if "Not Available" in testing:
                continue
            bedroom_type = rent_block.find('h4', class_ = "detailsLabel").text.strip().split(",")[0]
            print(bedroom_type, " ", float_price)
            if bedroom_type in rent_obj and rent_obj[bedroom_type] < float_price:
                continue
            else:
                rent_obj[bedroom_type] = float_price
            bath_number = rent_block.find('h4', class_ = "detailsLabel").findAll('span')[2].text
#             print('bath_number: ', bath_number)
            house_square_feet = rent_block.find('h4', class_ = "detailsLabel").findAll('span')[3].text
            square_feet = house_square_feet.split(" ")[0];
            print('house_square_feet: ', house_square_feet)
#             lease_length = rent_block.find('span', class_="leaseDepositLabel").text.split(',')[0].strip()
#             print('lease_length: ', lease_length)
        
        rent_obj["City_Rent"] = city
        rent_obj["Type"] = "Houses"
        rent_obj["State_Rent"] = state
        rent_obj["Zip_code"] = zip_code
        rent_obj["Full_Address"] = full_address
        rent_obj["Rent_link"] = find_tag
        rent_obj["Bath_Info"] = bath_number
        rent_obj["Square_Feet_Data"] = house_square_feet
        rent_obj["Square_Feet"] = square_feet
#         rent_obj["Lease_length"] = lease_length
        
        all_property.append(rent_obj)
#         print(all_property)
        time.sleep(2)
        
# pprint(all_property);

Page:  23
https://www.apartments.com/9018-keith-ave-west-hollywood-ca/872j9n0/
full_address:  9018 Keith Ave West Hollywood, CA 90069
1 bed   2000.0
house_square_feet:  5,260 sq ft
https://www.apartments.com/9018-keith-ave-west-hollywood-ca/jebk397/
full_address:  9018 Keith Ave West Hollywood, CA 90069
1 bed   2195.0
house_square_feet:  1,000 sq ft
Page:  24
https://www.apartments.com/9018-keith-ave-west-hollywood-ca/fstqz7t/
full_address:  9018 Keith Ave West Hollywood, CA 90069
1 bed   1920.0
house_square_feet:  800 sq ft
https://www.apartments.com/9018-keith-ave-west-hollywood-ca/6tqzw9v/
full_address:  9018 Keith Ave West Hollywood, CA 90069
1 bed   2275.0
house_square_feet:  750 sq ft
https://www.apartments.com/1538-11th-st-santa-monica-ca/h3ql070/
full_address:  1538 11th St Santa Monica, CA 90401
Studio   1895.0
house_square_feet:  450 sq ft
https://www.apartments.com/1445-stockton-st-st-helena-ca/gp8cfew/
full_address:  1445 Stockton St St. Helena, CA 94574
2 beds   7500.0
hou

https://www.apartments.com/57780-black-diamond-la-quinta-ca/gcc5t8z/
full_address:  57780 Black Diamond La Quinta, CA 92253
4 beds   7500.0
house_square_feet:  2,844 sq ft
https://www.apartments.com/68725-tachevah-dr-cathedral-city-ca/q81jpry/
full_address:  68725 Tachevah Dr Cathedral City, CA 92234
3 beds   4500.0
house_square_feet:  1,822 sq ft
https://www.apartments.com/19638-mountain-meadow-n-hidden-valley-lake-ca/qwkj3kx/
full_address:  19638 Mountain Meadow N Hidden Valley Lake, CA 95467
3 beds   2300.0
house_square_feet:  1,656 sq ft
Page:  26
https://www.apartments.com/22058-pacific-coast-hwy-malibu-ca/40pnb9k/
full_address:  22058 Pacific Coast Hwy Malibu, CA 90265
3 beds   50000.0
house_square_feet:  2,972 sq ft
https://www.apartments.com/34405-pacific-coast-hwy-malibu-ca/8yp6461/
full_address:  34405 Pacific Coast Hwy Malibu, CA 90265
4 beds   13200.0
house_square_feet:  2,989 sq ft
https://www.apartments.com/2122-rams-hill-dr-borrego-springs-ca/9mdm3h7/
full_address:  2122

https://www.apartments.com/143-waterford-dr-vacaville-ca/pt85y6v/
full_address:  143 Waterford Dr Vacaville, CA 95688
3 beds   2500.0
house_square_feet:  1,268 sq ft
https://www.apartments.com/60224-honeysuckle-st-la-quinta-ca/nm3ek0q/
full_address:  60224 Honeysuckle St La Quinta, CA 92253
2 beds   3500.0
house_square_feet:  1,359 sq ft
https://www.apartments.com/15968-snowcreek-canyon-rd-whitewater-ca/e7bqx6m/
full_address:  15968 Snowcreek Canyon Rd Whitewater, CA 92282
2 beds   1900.0
house_square_feet:  870 sq ft
https://www.apartments.com/75260-vista-corona-palm-desert-ca/r98twj0/
full_address:  75260 Vista Corona Palm Desert, CA 92211
2 beds   4000.0
house_square_feet:  1,038 sq ft
https://www.apartments.com/56408-palms-dr-la-quinta-ca/d5ck94k/
full_address:  56408 Palms Dr La Quinta, CA 92253
3 beds   7000.0
house_square_feet:  3,078 sq ft
Page:  28
https://www.apartments.com/61000-living-stone-dr-la-quinta-ca/m16p8vl/
full_address:  61000 Living Stone Dr La Quinta, CA 92253
2 

In [111]:
rent_data = pd.DataFrame(all_property)
rent_data

,1 bed,City_Rent,Type,State_Rent,Zip_code,Full_Address,Rent_link,Bath_Info,Square_Feet_Data,Square_Feet,Studio,2 beds,4 beds,3 beds,5 beds,10 beds,8 beds,7 beds,6 beds
0,2000.0,West Hollywood,Houses,CA,90069,"9018 Keith Ave West Hollywood, CA 90069",https://www.apartments.com/9018-keith-ave-west...,1 bath,"5,260 sq ft","5,260",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2195.0,West Hollywood,Houses,CA,90069,"9018 Keith Ave West Hollywood, CA 90069",https://www.apartments.com/9018-keith-ave-west...,1 bath,"1,000 sq ft","1,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1920.0,West Hollywood,Houses,CA,90069,"9018 Keith Ave West Hollywood, CA 90069",https://www.apartments.com/9018-keith-ave-west...,1 bath,800 sq ft,800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2275.0,West Hollywood,Houses,CA,90069,"9018 Keith Ave West Hollywood, CA 90069",https://www.apartments.com/9018-keith-ave-west...,1 bath,750 sq ft,750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Santa Monica,Houses,CA,90401,"1538 11th St Santa Monica, CA 90401",https://www.apartments.com/1538-11th-st-santa-...,1 bath,450 sq ft,450,1895.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,NaN,Belmont,Houses,CA,94002,"307 Hiller St Belmont, CA 94002",https://www.apartments.com/307-hiller-st-belmo...,2 baths,"1,290 sq ft","1,290",NaN,NaN,NaN,4500.0,NaN,NaN,NaN,NaN,NaN
123,NaN,Riverbank,Houses,CA,95367,"5639 Suttermill Dr Riverbank, CA 95367",https://www.apartments.com/5639-suttermill-dr-...,2.5 baths,"3,050 sq ft","3,050",NaN,NaN,2500.0,NaN,NaN,NaN,NaN,NaN,NaN
124,NaN,La Jolla,Houses,CA,92037,"2075 Torrey Pines Rd La Jolla, CA 92037",https://www.apartments.com/2075-torrey-pines-r...,4 baths,"4,000 sq ft","4,000",NaN,NaN,NaN,NaN,15000.0,NaN,NaN,NaN,NaN
125,NaN,Frazier Park,Houses,CA,93225,"1221 Snowbird Dr Frazier Park, CA 93225",https://www.apartments.com/1221-snowbird-dr-fr...,2 baths,"2,400 sq ft","2,400",NaN,NaN,1799.0,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
# Export file as a CSV, without the Pandas index, but with the header
rent_data.to_csv("Output/Apartment.com/Q2-2021_Apartment.com_Houses_Data_page23-28.csv", index=False, header=True)